# Загрузка данных и аналитика

In [ ]:
pip install scikit-learn optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.3 MB/s eta 0:00:00


In [ ]:
pip install torchinfo

In [ ]:
pip install tensorflow

In [ ]:
pip install keras

In [ ]:
# импортруем библиотеки
import pandas as pd
from sklearn.metrics import classification_report, roc_curve, auc, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import optuna
import torch
from torch.utils.data import DataLoader
import torchinfo
import gc
import sys
import transformers
from transformers import pipeline
from transformers import AutoModel
from transformers import AutoTokenizer
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from transformers import AdamW, get_linear_schedule_with_warmup
transformers.logging.set_verbosity('CRITICAL')
from tqdm import tqdm
tqdm.pandas()
from sklearn.model_selection import train_test_split
import os
import pickle
from sklearn.metrics import classification_report, roc_curve, auc, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping



In [ ]:
# подключаюсь к диску
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
# загрузка данных
#comments =  pd.read_excel("comment_copy_copy_copy.xlsx")

In [ ]:
comments = comment_copy_copy_copy

In [ ]:
print(f"Размер датасета: {comments.shape}")

Размер датасета: (1344, 2)


In [ ]:
comments.rename(columns={'Категория': 'topic'}, inplace=True)

In [ ]:
comments.topic.value_counts(normalize=True)

,proportion
topic,
Вопрос по лечению,0.377976
Аналогичный опыт,0.139881
Вопрос по диагнозу,0.107887
Вопрос по диагностике,0.104911
Поиск врача/клиники,0.071429
Питание и образ жизни,0.052827
"Гос. поддержка, фонды, льготы, инвалидность",0.046131
Другое,0.030506
Дефицит препаратов,0.029018


# TF-IDF + SVM

In [ ]:
# Разделение данных
X = comments['lemmatized_text']  # текстовые сообщения
y = comments['topic']    # темы сообщений

# Разделение на обучающую и тестовую выборки (70% - 30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
# Создание объекта векторизатора
tfidf_vectorizer = TfidfVectorizer()

# Применение TF-IDF к обучающему набору
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Применение TF-IDF к тестовому набору
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [ ]:
# Функция определения гиперпараметров

def objective(trial):
    # Подбор гиперпараметров
    tfidf_max_features = trial.suggest_int('max_features', 1000, 10000)
    tfidf_ngram_range = trial.suggest_categorical('ngram_range', [(1, 1), (1, 2), (2, 2)])

    # Преобразование текста в TF-IDF
    vectorizer = TfidfVectorizer(max_features=tfidf_max_features, ngram_range=tfidf_ngram_range)
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)

    # Гиперпараметры для SVM
    svm_c = trial.suggest_float('C', 1e-5, 1e5, log=True)
    svm_kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])

    # Создание и обучение модели SVM
    model = SVC(C=svm_c, kernel=svm_kernel)
    model.fit(X_train_tfidf, y_train)

    # Прогнозирование и расчет точности
    y_pred = model.predict(X_test_tfidf)
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy


In [ ]:
# Запуск оптимизации
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # Задайте нужное количество испытаний

# Вывод наилучших гиперпараметров
print("Best hyperparameters: ", study.best_params)
print("Best accuracy: ", study.best_value)

[I 2024-12-20 20:15:39,310] A new study created in memory with name: no-name-d74b3989-c80e-4975-93ec-6f6ad5c209b7
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (1, 1) which is of type tuple.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (1, 2) which is of type tuple.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (2, 2) which is of type tuple.
  warnings.warn(message)
[I 2024-12-20 20:15:39,799] Trial 0 finished with value: 0.3787128712871287 and parameters:

Best hyperparameters:  {'max_features': 9973, 'ngram_range': (1, 2), 'C': 43.77140799797923, 'kernel': 'linear'}
Best accuracy:  0.5371287128712872


In [ ]:
# Обучение модели с наилучшими гиперпараметрами
best_params = study.best_params
final_vectorizer = TfidfVectorizer(max_features=best_params['max_features'], ngram_range=best_params['ngram_range'])
X_train_final = final_vectorizer.fit_transform(X_train)
X_test_final = final_vectorizer.transform(X_test)

SVM_model = SVC(C=best_params['C'], kernel=best_params['kernel'])
SVM_model.fit(X_train_final, y_train)



SVC(C=43.77140799797923, kernel='linear')

In [ ]:
# обучение тестовой выборки
y_final_pred = SVM_model.predict(X_test_final)


In [ ]:
# Расчет метрик
accuracy_SVM = accuracy_score(y_test, y_final_pred)
precision_SVM = precision_score(y_test, y_final_pred, average='weighted')
recall_SVM = recall_score(y_test, y_final_pred, average='weighted')
f1_SVM = f1_score(y_test, y_final_pred, average='weighted')

print(f'Accuracy: {accuracy_SVM* 100 :.2f}%')
print(f'Precision: {precision_SVM* 100 :.2f}%')
print(f'Recall: {recall_SVM* 100 :.2f}%')
print(f'F1: {f1_SVM* 100 :.2f}%')

Accuracy: 53.71%
Precision: 51.63%
Recall: 53.71%
F1: 48.72%


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Сохраняем модель в pkl файл
model_filename = '/content/drive/MyDrive/Colab Notebooks/SVM_model.pkl'

with open(model_filename, 'wb') as file:
    pickle.dump(SVM_model, file)

print("Модель успешно сохранена в", model_filename)

Модель успешно сохранена в /content/drive/MyDrive/Colab Notebooks/SVM_model.pkl


# BERT

In [ ]:
#

idx_to_sentiment = {k: v for k, v in enumerate(comments['topic'].unique())}
sentiment_to_idx = {v: k for k, v in idx_to_sentiment.items()}
comments['label'] = comments['topic'].apply(lambda x: sentiment_to_idx[x])
comments['label'] = comments['label'].apply(lambda x: torch.tensor(x))
comments.sample(3, random_state=0)

,topic,lemmatized_text,label
1340,Аналогичный опыт,подсказывать стоматит лечить полоскание помога...,tensor(8)
328,Другое,коллега онколог привет москва проходить стоящи...,tensor(9)
1151,Вопрос по диагностике,посоветовать делатьповышать тромбоцит лейкоци...,tensor(4)


In [ ]:
# функция токенизации
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def get_tokens(text: str):

    encoded_dict = tokenizer.encode_plus(text,
                                         add_special_tokens=True,
                                         truncation=True,
                                         max_length=128,
                                         padding='max_length',
                                         return_attention_mask=True,
                                         return_tensors='pt')
    input_ids = encoded_dict['input_ids'].squeeze(0)
    attention_mask = encoded_dict['attention_mask'].squeeze(0)
    return input_ids, attention_mask

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
comments['input_ids'], comments['attention_mask'] = zip(*comments['lemmatized_text'].progress_apply(get_tokens))
comments.sample(3, random_state=0)

100%|██████████| 1344/1344 [00:01<00:00, 915.36it/s]


,topic,lemmatized_text,label,input_ids,attention_mask
1340,Аналогичный опыт,подсказывать стоматит лечить полоскание помога...,tensor(8),"[tensor(101), tensor(1194), tensor(14150), ten...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
328,Другое,коллега онколог привет москва проходить стоящи...,tensor(9),"[tensor(101), tensor(1189), tensor(14150), ten...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
1151,Вопрос по диагностике,посоветовать делатьповышать тромбоцит лейкоци...,tensor(4),"[tensor(101), tensor(1194), tensor(14150), ten...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."


In [ ]:
train_split, test_split = train_test_split(comments, test_size=0.2, random_state=0)

In [ ]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, data_df: pd.DataFrame):

        self.data_df = data_df

    def __len__(self):

        data_len = self.data_df.shape[0]
        return data_len

    def __getitem__(self, idx: int):

        input_ids = self.data_df.iloc[idx]['input_ids']
        attention_mask = self.data_df.iloc[idx]['attention_mask']
        label = self.data_df.iloc[idx]['label']
        return input_ids, attention_mask, label

In [ ]:
train_dataset = Dataset(train_split)
train_loader = DataLoader(train_dataset,
                          shuffle=True,
                          batch_size=32,
                          drop_last=True)

test_dataset = Dataset(test_split)
test_loader = DataLoader(test_dataset,
                        shuffle=False,
                        batch_size=32,
                        drop_last=False)

In [ ]:
BERT_model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                      num_labels=len(sentiment_to_idx),
                                                      output_attentions=False,
                                                      output_hidden_states=False)
torchinfo.summary(BERT_model, input_data=torch.randint(low=0, high=30522, size=(1, 128)))

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Layer (type:depth-idx)                                       Output Shape              Param #
BertForSequenceClassification                                [1, 13]                   --
├─BertModel: 1-1                                             [1, 768]                  --
│    └─BertEmbeddings: 2-1                                   [1, 128, 768]             --
│    │    └─Embedding: 3-1                                   [1, 128, 768]             23,440,896
│    │    └─Embedding: 3-2                                   [1, 128, 768]             1,536
│    │    └─Embedding: 3-3                                   [1, 128, 768]             393,216
│    │    └─LayerNorm: 3-4                                   [1, 128, 768]             1,536
│    │    └─Dropout: 3-5                                     [1, 128, 768]             --
│    └─BertEncoder: 2-2                                      [1, 128, 768]             --
│    │    └─ModuleList: 3-6                                  --             

In [ ]:
epochs = 3
training_steps = len(train_loader) * epochs
optimizer = AdamW(BERT_model.parameters(), lr=5e-5, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=20,
                                            num_training_steps=training_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
class RunningAverage:

    def __init__(self) -> None:
        self.count = 0
        self.total = 0.0

    def update(self, n: float) -> None:

        self.total += n
        self.count += 1

    def __call__(self) -> float:

        running_avg = self.total / (self.count + 1e-15)
        return running_avg

In [ ]:
device_cpu = torch.device('cpu')
BERT_model.to(device_cpu)

for e in range(epochs):
    loss_avg = RunningAverage()
    val_avg = RunningAverage()
    val_loss_avg = RunningAverage()

    # Force the garbage collection
    torch.cuda.empty_cache()
    gc.collect()

    BERT_model.train()
    with tqdm(total=len(train_loader), leave=False, file=sys.stdout) as t:
        t.set_description(f'Epoch {e + 1}')

        for batch_n, batch_data in enumerate(train_loader):
            # Load the current batch data on the defined device
            b_input_ids = batch_data[0].to(device_cpu)
            b_input_mask = batch_data[1].to(device_cpu)
            b_labels = batch_data[2].to(device_cpu)

            BERT_model.zero_grad()
            model_output = BERT_model(b_input_ids,
                                 token_type_ids=None,
                                 attention_mask=b_input_mask,
                                 labels=b_labels)
            loss, logits = model_output['loss'], model_output['logits']
            loss_avg.update(loss.item())
            loss.backward()
            # Adaptively clip gradients to avoid gradient explosion
            torch.nn.utils.clip_grad_norm_(BERT_model.parameters(), 2.0)
            optimizer.step()
            # Update the LR scheduler every step
            scheduler.step()

            t.set_postfix({'stats': f'train_loss: {loss_avg():.4f}'})
            t.update()
            stats_time_elapsed = t.format_interval(t.format_dict['elapsed'])

    # Evaluate every epoch
    BERT_model.eval()
    with torch.no_grad():
        for val_batch_data in test_loader:
            # Load the current batch data on the defined device
            b_val_input_ids = val_batch_data[0].to(device_cpu)
            b_val_input_mask = val_batch_data[1].to(device_cpu)
            b_val_labels = val_batch_data[2].to(device_cpu)

            val_model_output = BERT_model(b_val_input_ids,
                                     token_type_ids=None,
                                     attention_mask=b_val_input_mask,
                                     labels=b_val_labels)
            val_loss, val_logits = val_model_output['loss'], val_model_output['logits']
            val_loss_avg.update(val_loss.item())
            val_logits = torch.argmax(val_logits, 1)
            # Calculate the validation accuracy
            val_batch = torch.sum(val_logits == b_val_labels) / b_val_labels.shape[0]
            val_avg.update(val_batch.item())

    # Print the epoch stats
    print(f'Epoch {e + 1}. Train_loss: {loss_avg():.4f},'
          f' val_loss: {val_loss_avg():.4f}, val: {val_avg():.4f}, time: {stats_time_elapsed}')

Epoch 1. Train_loss: 2.1949, val_loss: 2.0652, val: 0.3622, time: 25:38
Epoch 2. Train_loss: 1.9969, val_loss: 2.0485, val: 0.3622, time: 24:24
Epoch 3. Train_loss: 1.9750, val_loss: 2.0432, val: 0.3622, time: 24:09


In [ ]:
# Преобразуем в numpy массивы для оценки
predictions = np.array(val_logits)
true_labels = np.array(b_val_labels)

In [ ]:
# Расчет метрик
accuracy_BERT = accuracy_score(true_labels, predictions)
precision_BERT = precision_score(true_labels, predictions, average='weighted')
recall_BERT = recall_score(true_labels, predictions, average='weighted')
f1_BERT = f1_score(true_labels, predictions, average='weighted')

print(f'Accuracy: {accuracy_BERT* 100 :.2f}%')
print(f'Precision: {precision_BERT* 100 :.2f}%')
print(f'Recall: {recall_BERT* 100 :.2f}%')
print(f'F1: {f1_BERT* 100 :.2f}%')

Accuracy: 38.46%
Precision: 14.79%
Recall: 38.46%
F1: 21.37%


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Сохраняем модель в pkl файл
model_filename = '/content/drive/MyDrive/Colab Notebooks/BERT_model.pkl'

with open(model_filename, 'wb') as file:
    pickle.dump(BERT_model, file)

print("Модель успешно сохранена в", model_filename)

Модель успешно сохранена в /content/drive/MyDrive/Colab Notebooks/BERT_model.pkl


# LSTM

In [ ]:
# Подготовка данных
X = comments['lemmatized_text'].values
y = comments['topic'].values

# Преобразование меток в числовой формат
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Токенизация
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

# Устанавливаем максимальную длину последовательностей
max_len = max(len(seq) for seq in sequences)
X = pad_sequences(sequences, maxlen=max_len)

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Создание модели LSTM
model_lstm = keras.Sequential([
    layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_len),
    layers.LSTM(128),
    layers.Dense(64, activation='relu'),
    layers.Dense(13, activation='softmax')
])

model_lstm.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
# Добавление ранней остановки
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
 # Обучение модели
history = model_lstm.fit(X_train, y_train,
                         validation_split=0.2,
                         epochs=50,
                         batch_size=32,
                         callbacks=[early_stopping])

Epoch 1/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 24s 616ms/step - accuracy: 0.2953 - loss: 2.3860 - val_accuracy: 0.3535 - val_loss: 2.0317
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 22s 688ms/step - accuracy: 0.3712 - loss: 2.0588 - val_accuracy: 0.3535 - val_loss: 2.0209
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 19s 688ms/step - accuracy: 0.4005 - loss: 1.9640 - val_accuracy: 0.3535 - val_loss: 2.0306
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 20s 660ms/step - accuracy: 0.4080 - loss: 1.8412 - val_accuracy: 0.3535 - val_loss: 1.9083
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 16s 502ms/step - accuracy: 0.4977 - loss: 1.4942 - val_accuracy: 0.3581 - val_loss: 1.9093
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 19s 444ms/step - accuracy: 0.6909 - loss: 1.0046 - val_accuracy: 0.4186 - val_loss: 2.0782
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 9s 318ms/step - accuracy: 0.8441 - loss: 0.5627 - val_accuracy: 0.4186 - val_loss: 2.1691
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 12s 426ms/step - accuracy: 0.8781 - loss: 0.4708 - val_accur

In [ ]:
# Прогнозы на тестовом наборе
y_pred = model_lstm.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 206ms/step


In [ ]:
# Расчет метрик
accuracy_LSTM = accuracy_score(y_test, y_pred_classes)
precision_LSTM = precision_score(y_test, y_pred_classes, average='weighted')
recall_LSTM = recall_score(y_test, y_pred_classes, average='weighted')
f1_LSTM = f1_score(y_test, y_pred_classes, average='weighted')

print(f'Accuracy: {accuracy_LSTM* 100 :.2f}%')
print(f'Precision: {precision_LSTM* 100 :.2f}%')
print(f'Recall: {recall_LSTM* 100 :.2f}%')
print(f'F1: {f1_LSTM* 100 :.2f}%')

Accuracy: 42.75%
Precision: 24.04%
Recall: 42.75%
F1: 30.72%


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Сохраняем модель в pkl файл
model_filename = '/content/drive/MyDrive/Colab Notebooks/model_lstm.pkl'

with open(model_filename, 'wb') as file:
    pickle.dump(model_lstm, file)

print("Модель успешно сохранена в", model_filename)

Модель успешно сохранена в /content/drive/MyDrive/Colab Notebooks/model_lstm.pkl


# GRU

In [ ]:
# Подготовка данных
texts = comments['lemmatized_text'].values
labels = comments['topic'].values

# Преобразование меток в числовой формат
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Разделение на обучающую и тестовую выборки
x_train, x_test, y_train, y_test = train_test_split(texts, labels_encoded, test_size=0.2, random_state=42)

In [ ]:
# Токенизация текста
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(x_train)
x_train_seq = tokenizer.texts_to_sequences(x_train)
x_test_seq = tokenizer.texts_to_sequences(x_test)

# Установка максимальной длины последовательности
max_length = max(len(seq) for seq in x_train_seq)

# Паддинг последовательностей
x_train_padded = pad_sequences(x_train_seq, maxlen=max_length, padding='post')
x_test_padded = pad_sequences(x_test_seq, maxlen=max_length, padding='post')

In [ ]:
# Создание модели
model_gru = Sequential()
model_gru.add(Embedding(input_dim=10000, output_dim=128, input_length=max_length))
model_gru.add(GRU(64, return_sequences=False))
model_gru.add(Dropout(0.5))
model_gru.add(Dense(len(np.unique(labels_encoded)), activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
# Компиляция модели
model_gru.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Ранняя остановка
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
# Обучение модели
history = model_gru.fit(x_train_padded, y_train, validation_split=0.2, epochs=20, batch_size=32,
                    callbacks=[early_stopping])

Epoch 1/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 16s 448ms/step - accuracy: 0.3125 - loss: 2.3641 - val_accuracy: 0.3535 - val_loss: 2.0668
Epoch 2/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 8s 285ms/step - accuracy: 0.3858 - loss: 2.0747 - val_accuracy: 0.3535 - val_loss: 2.0169
Epoch 3/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 304ms/step - accuracy: 0.3729 - loss: 2.0191 - val_accuracy: 0.3535 - val_loss: 2.0237
Epoch 4/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 351ms/step - accuracy: 0.3766 - loss: 2.0506 - val_accuracy: 0.3535 - val_loss: 2.0187
Epoch 5/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 10s 326ms/step - accuracy: 0.3552 - loss: 2.0850 - val_accuracy: 0.3535 - val_loss: 2.0138
Epoch 6/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 12s 389ms/step - accuracy: 0.3527 - loss: 2.0735 - val_accuracy: 0.3535 - val_loss: 2.0154
Epoch 7/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 20s 374ms/step - accuracy: 0.3784 - loss: 2.0189 - val_accuracy: 0.3535 - val_loss: 2.0123
Epoch 8/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 10s 380ms/step - accuracy: 0.3969 - loss: 2.0064 - val_accur

In [ ]:
# Оценка модели на тестовых данных
y_pred = model_gru.predict(x_test_padded)
y_pred_classes = np.argmax(y_pred, axis=1)

9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 271ms/step


In [ ]:
# Расчет метрик
accuracy_gru = accuracy_score(y_test, y_pred_classes)
precision_gru = precision_score(y_test, y_pred_classes, average='weighted')
recall_gru = recall_score(y_test, y_pred_classes, average='weighted')
f1_gru = f1_score(y_test, y_pred_classes, average='weighted')

print(f'Accuracy: {accuracy_gru* 100 :.2f}%')
print(f'Precision: {precision_gru* 100 :.2f}%')
print(f'Recall: {recall_gru* 100 :.2f}%')
print(f'F1: {f1_gru* 100 :.2f}%')

Accuracy: 40.89%
Precision: 16.72%
Recall: 40.89%
F1: 23.74%


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Сохраняем модель в pkl файл
model_filename = '/content/drive/MyDrive/Colab Notebooks/model_gru.pkl'

with open(model_filename, 'wb') as file:
    pickle.dump(model_gru, file)

print("Модель успешно сохранена в", model_filename)

# Сравнение моделей

In [ ]:
# Соберем основные метрики по моделям в одну таблицу
regression_df = pd.DataFrame(
    data = [
        [f'{accuracy_SVM* 100 :.2f}%', f'{accuracy_LSTM* 100 :.2f}%', f'{accuracy_gru* 100 :.2f}%', f'{accuracy_BERT* 100 :.2f}%'],
        [f'{precision_SVM* 100 :.2f}%', f'{precision_LSTM* 100 :.2f}%', f'{precision_gru* 100 :.2f}%', f'{precision_BERT* 100 :.2f}%'],
        [f'{recall_SVM* 100 :.2f}%', f'{recall_LSTM* 100 :.2f}%', f'{recall_gru* 100 :.2f}%', f'{recall_BERT* 100 :.2f}%'],
        [f'{f1_SVM* 100 :.2f}%', f'{f1_LSTM* 100 :.2f}%', f'{f1_gru* 100 :.2f}%', f'{f1_gru* 100 :.2f}%']
    ],
    columns= ['SVM', 'LSTM', 'GRU', 'BERT'],
    index = ['Accuracy', 'Precision', 'Recall', 'F1']
)
display(regression_df)

,SVM,LSTM,GRU,BERT
Accuracy,53.71%,42.75%,40.89%,38.46%
Precision,51.63%,24.04%,16.72%,14.79%
Recall,53.71%,42.75%,40.89%,38.46%
F1,48.72%,30.72%,23.74%,23.74%
